# Tag, retrieve, and save results

<details>
<summary><b>Package versions</b></summary>

The code on this page was developed using the following requirements.
We recommend using these versions or newer.

```
qiskit-ibm-runtime~=0.40.1
```
</details>

This guide focuses on best practices in retrieving and managing your job results, so that you can better organize, track, and understand your experiments. 

## Add and search by job tags

You can assign one or more tags to your jobs when you run them so that you can filter by a tag later. You might want to use job tags to label particular error mitigation settings, circuit parameters, and so forth.

In [1]:
from qiskit_ibm_runtime import QiskitRuntimeService, Sampler
from qiskit import QuantumCircuit
from qiskit.transpiler import generate_preset_pass_manager

service = QiskitRuntimeService()
backend = service.least_busy(simulator=False, operational=True)

qc = QuantumCircuit(2)
qc.h(0)
qc.cx(0, 1)
qc.measure_all()

sampler = Sampler(backend)

pm = generate_preset_pass_manager(backend=backend, optimization_level=1)
isa_circuit = pm.run(qc)

# Assign tags before executing
sampler.options.environment.job_tags = ["experiment-2025", "sampler-example"]

# Submit
job = sampler.run([isa_circuit])

print(service.job(job.job_id()).tags)

['experiment-2025', 'sampler-example']


You can add tags after you submit a job with the `update_tags()` method. This method overwrites current tags, so if you have already assigned tags to a job and you want to add additional tags, be sure to re-assign the original tags.

In [15]:
# add a new tag while keeping the previously assigned tags
job.update_tags(["experiment-2025", "sampler-example", "127-qubit"])

# Confirm updated tags
print(job.tags)

['sampler-example', 'experiment-2025', '127-qubit']


## Retrieve jobs from IBM Quantum

IBM Quantum automatically stores results from every job for you to retrieve at a later date. Use this feature to continue quantum programs across kernel restarts and review past results. You can get the ID of a job programmatically through its `job_id` method, or you can see all your submitted jobs and their IDs on the [Workloads page](https://quantum.cloud.ibm.com/workloads).

To find a job programmatically, use the [`QiskitRuntimeService.jobs`](/docs/api/qiskit-ibm-runtime/qiskit-runtime-service#jobs) method. By default, this returns the most recent jobs, but you can also filter jobs by backend name, creation date, and more. The following cell finds any jobs submitted in the last three months. The `created_after` argument must be a [`datetime.datetime`](https://docs.python.org/3.8/library/datetime.html#datetime.datetime) object.

In [1]:
import datetime
from qiskit_ibm_runtime import QiskitRuntimeService

three_months_ago = datetime.datetime.now() - datetime.timedelta(days=90)

service = QiskitRuntimeService()
jobs_in_last_three_months = service.jobs(created_after=three_months_ago)
jobs_in_last_three_months[:3]  # show first three jobs

[<RuntimeJobV2('d2n1d2cg59ks73c6vu3g', 'sampler')>,
 <RuntimeJobV2('d2n18dehb60s73cvalg0', 'sampler')>,
 <RuntimeJobV2('d2ff2cms6qcs738f4lb0', 'sampler')>]

You can also select by backend, job state, session, and more. For more information, see [`QiskitRuntimeService.jobs`](/docs/api/qiskit-ibm-runtime/qiskit-runtime-service#jobs) in the API documentation.

Once you have the job ID, use the [`QiskitRuntimeService.job`](/docs/api/qiskit-ibm-runtime/qiskit-runtime-service#job) method to retrieve it.

In [2]:
# Get ID of most recent successful job for demonstration.
# This will not work if you've never successfully run a job.
successful_job = next(
    j for j in service.jobs(limit=1000) if j.status() == "DONE"
)
job_id = successful_job.job_id()
print(job_id)

d2n1d2cg59ks73c6vu3g


In [3]:
retrieved_job = service.job(job_id)
retrieved_job.result()

PrimitiveResult([SamplerPubResult(data=DataBin(c=BitArray(<shape=(), num_shots=4096, num_bits=2>)), metadata={'circuit_metadata': {}}), SamplerPubResult(data=DataBin(c=BitArray(<shape=(), num_shots=4096, num_bits=2>)), metadata={'circuit_metadata': {}}), SamplerPubResult(data=DataBin(c=BitArray(<shape=(), num_shots=4096, num_bits=2>)), metadata={'circuit_metadata': {}}), SamplerPubResult(data=DataBin(c=BitArray(<shape=(), num_shots=4096, num_bits=2>)), metadata={'circuit_metadata': {}}), SamplerPubResult(data=DataBin(c=BitArray(<shape=(), num_shots=4096, num_bits=2>)), metadata={'circuit_metadata': {}}), SamplerPubResult(data=DataBin(c=BitArray(<shape=(), num_shots=4096, num_bits=2>)), metadata={'circuit_metadata': {}}), SamplerPubResult(data=DataBin(c=BitArray(<shape=(), num_shots=4096, num_bits=2>)), metadata={'circuit_metadata': {}}), SamplerPubResult(data=DataBin(c=BitArray(<shape=(), num_shots=4096, num_bits=2>)), metadata={'circuit_metadata': {}}), SamplerPubResult(data=DataBin(c

<CodeAssistantAdmonition
  tagLine="Always forgetting how to retrieve a job? Try this prompt with Qiskit Code Assistant:"
  prompts={[
    "# Retrieve job JOB_ID from IBM Runtime and print the result"
  ]}
/>

## Save results to disk

Quantum workflows often take a while to complete and can run over many sessions. Restarting your Python kernel means you'll lose any results stored in memory. To avoid loss of data, you can save results to disk. To do this, use Python's built-in JSON library with Qiskit Runtime's encoders.

In [4]:
import json
from qiskit_ibm_runtime import RuntimeEncoder

with open("result.json", "w") as file:
    json.dump(retrieved_job.result(), file, cls=RuntimeEncoder)

You can then load this array from disk in a separate kernel.

In [5]:
from qiskit_ibm_runtime import RuntimeDecoder

with open("result.json", "r") as file:
    result = json.load(file, cls=RuntimeDecoder)

result

PrimitiveResult([SamplerPubResult(data=DataBin(c=BitArray(<shape=(), num_shots=4096, num_bits=2>)), metadata={'circuit_metadata': {}}), SamplerPubResult(data=DataBin(c=BitArray(<shape=(), num_shots=4096, num_bits=2>)), metadata={'circuit_metadata': {}}), SamplerPubResult(data=DataBin(c=BitArray(<shape=(), num_shots=4096, num_bits=2>)), metadata={'circuit_metadata': {}}), SamplerPubResult(data=DataBin(c=BitArray(<shape=(), num_shots=4096, num_bits=2>)), metadata={'circuit_metadata': {}}), SamplerPubResult(data=DataBin(c=BitArray(<shape=(), num_shots=4096, num_bits=2>)), metadata={'circuit_metadata': {}}), SamplerPubResult(data=DataBin(c=BitArray(<shape=(), num_shots=4096, num_bits=2>)), metadata={'circuit_metadata': {}}), SamplerPubResult(data=DataBin(c=BitArray(<shape=(), num_shots=4096, num_bits=2>)), metadata={'circuit_metadata': {}}), SamplerPubResult(data=DataBin(c=BitArray(<shape=(), num_shots=4096, num_bits=2>)), metadata={'circuit_metadata': {}}), SamplerPubResult(data=DataBin(c